# Extracting training data from the ODC

* **Products used:** 
[gm_s2_semiannual](https://explorer.digitalearth.africa/gm_s2_semiannual)


## Description
This notebook will extract training data over northern Africa using geometries within a shapefile (or geojson). To do this, we rely on a custom `deafrica-sandbox-notebooks` function called `collect_training_data`, contained within the [deafrica_tools.classification](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/minty-fresh-sandbox/Tools/deafrica_tools/classification.py) script.

1. Import, and preview our training data contained in the file: `'data/Northern_training_data_YYYYMMDD.geojson'`
2. Extract training data from the datacube using a custom defined feature layer function that we can pass to `collect_training_data`. The training data function is stored in the python file `feature_layer_functions.py` - the functions are stored in a seperate file simply to keep this notebook tidy.

    - **The features used to create the cropland mask are as follows:**
        - For two seasons, January to June, and July to Decemeber:
            - A geomedian composite of nine Sentinel-2 spectral bands
            - Three measures of median absolute deviation
            - NDVI, MNDWI, and LAI
            - Cumulative Rainfall from CHIRPS
        - Slope from SRTM (not seasonal, obviously)
            
3. Separate the coordinate values in the returned training data from step 2, and export the coordinates as a text file.
4. Export the remaining training data (features other than coordinates) to disk as a text file for use in subsequent scripts

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

## Load packages


In [1]:
%matplotlib inline

import os
import warnings
warnings.filterwarnings("ignore")

import datacube
import numpy as np
import xarray as xr
import geopandas as gpd
from odc.io.cgroups import get_cpu_quota
from datacube.utils.geometry import assign_crs
from datacube.utils.rio import configure_s3_access
configure_s3_access(aws_unsigned=True, cloud_defaults=True)

from deafrica_tools.plotting import map_shapefile
from deafrica_tools.classification import collect_training_data 

#import the custom feature layer functions
from feature_layer_functions import gm_mads_two_seasons_training

## Analysis parameters

* `path`: The path to the input shapefile from which we will extract training data.
* `field`: This is the name of column in your shapefile attribute table that contains the class labels. **The class labels must be integers**


In [2]:
path = 'data/sahel_training_data_20211011.geojson'

output_suffix = '20211011'

field = 'Class'

### Automatically find the number of cpus

> **Note**: With supervised classification, its common to have many, many labelled geometries in the training data. `collect_training_data` can parallelize across the geometries in order to speed up the extracting of training data. Setting `ncpus>1` will automatically trigger the parallelization, however, its best to set `ncpus=1` to begin with to assist with debugging before triggering the parallelization. 

In [3]:
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

ncpus = 15


## Load & preview polygon data

We can load and preview our input data shapefile using `geopandas`. The shapefile should contain a column with class labels (e.g. 'class'). These labels will be used to train our model. 

> Remember, the class labels **must** be represented by `integers`.


In [4]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

,smpl_class,Class,geometry
0,1,0,"POLYGON ((46.86733 3.59059, 46.86774 3.59059, ..."
1,2,1,"POLYGON ((32.77438 12.95660, 32.77479 12.95660..."
2,1,0,"POLYGON ((18.03356 22.03910, 18.03398 22.03910..."
3,1,0,"POLYGON ((33.41577 9.73345, 33.41619 9.73345, ..."
4,1,0,"POLYGON ((-3.03499 23.25667, -3.03457 23.25667..."


In [6]:
# Plot training data in an interactive map
# map_shapefile(input_data, attribute=field)

Now, we can pass this shapefile to `collect_training_data`.  For each of the geometries in our shapefile we will extract features in accordance with the function `feature_layer_functions.gm_mads_two_seasons_training`.

First, we need to set up a few extra inputs for `collect_training_data` and the datacube.  See the function docs [here](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/03b7b41d5f6526ff3f33618f7a0b48c0d10a155f/Scripts/deafrica_classificationtools.py#L650) for more information on these parameters.



In [7]:
#set up our inputs to collect_training_data
zonal_stats = 'median'

# Set up the inputs for the ODC query
time = ('2019')
measurements = [
        "blue",
        "green",
        "red",
        "nir",
        "swir_1",
        "swir_2",
        "red_edge_1",
        "red_edge_2",
        "red_edge_3",
        "bcdev",
        "edev",
        "sdev"
]
resolution = (-10, 10)
output_crs = 'epsg:6933'

In [8]:
#generate a new datacube query object
query = {
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'output_crs': output_crs,
    'group_by' : 'solar_day',
    'resampling': 'bilinear'
}

## Extract training data

> Remember, if running this function for the first time, its advisable to set `ncpus=1` to assist with debugging before triggering the parallelization (which won't return errors if something is not working correctly).  You can also limit the number of polygons to run for the first time by passing in `gdf=input_data[0:5]`, for example.

In [9]:
%%time
warnings.filterwarnings("ignore")
column_names, model_input = collect_training_data(
                                    gdf=input_data,
                                    dc_query=query,
                                    ncpus=15,
                                    field=field,
                                    zonal_stats=zonal_stats,
                                    fail_threshold=0.0075,
                                    feature_func=gm_mads_two_seasons_training
                                    )

Taking zonal statistic: median


  0%|          | 0/1365 [00:00<?, ?it/s]

Percentage of possible fails after run 1 = 0.0 %
Removed 6 rows wth NaNs &/or Infs
Output shape:  (1359, 36)
CPU times: user 7.87 s, sys: 955 ms, total: 8.82 s
Wall time: 9min 23s


In [10]:
print(column_names)
print('')
print(np.array_str(model_input, precision=2, suppress_small=True))

['Class', 'blue_S1', 'green_S1', 'red_S1', 'nir_S1', 'swir_1_S1', 'swir_2_S1', 'red_edge_1_S1', 'red_edge_2_S1', 'red_edge_3_S1', 'bcdev_S1', 'edev_S1', 'sdev_S1', 'NDVI_S1', 'LAI_S1', 'MNDWI_S1', 'rain_S1', 'blue_S2', 'green_S2', 'red_S2', 'nir_S2', 'swir_1_S2', 'swir_2_S2', 'red_edge_1_S2', 'red_edge_2_S2', 'red_edge_3_S2', 'bcdev_S2', 'edev_S2', 'sdev_S2', 'NDVI_S2', 'LAI_S2', 'MNDWI_S2', 'rain_S2', 'slope', 'x_coord', 'y_coord']

[[      0.         0.19       0.31 ...       4.31 -292815.   2888155.  ]
 [      1.         0.13       0.18 ...       3.17 3361845.   1849475.  ]
 [      0.         0.23       0.38 ...       6.9  1112570.   2322935.  ]
 ...
 [      0.         0.05       0.08 ...       5.34 2923125.    602705.  ]
 [      0.         0.18       0.27 ...       0.71 4686355.   1327975.  ]
 [      0.         0.13       0.19 ...       2.46 4497055.    425185.  ]]


## Export training data

Once we've collected all the training data we require, we can write the data to disk. This will allow us to import the data in the next step(s) of the workflow.


In [17]:
#set the name and location of the output file
output_file = "results/training_data/sahel_training_data_"+output_suffix+".txt"

In [18]:
#grab all columns except the x-y coords
model_col_indices = [column_names.index(var_name) for var_name in column_names]
#Export files to disk
np.savetxt(output_file, model_input[:, model_col_indices], header=" ".join(column_names), fmt="%4f")

## Next steps

To continue working through the notebooks in this `northern Africa Cropland Mask` workflow, go to the next notebook `2_Inspect_training_data.ipynb`.

1. **Extract_training_data (this notebook)** 
2. [Inspect_training_data](2_Inspect_training_data.ipynb)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** Dec 2020
